In [51]:
import time
import torch
import torch.nn as nn
import numpy as np
from model import GPT, GPTConfig
from context_free_grammar import CFG
import wandb
import lightning.pytorch as pl
import pytorch_lightning

In [52]:
wandb.login()

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


True

In [53]:
cfg = CFG(L=3, ns=[1, 3, 3, 10], nr=[2, 2, 2], T=[8, 8, 8])

In [54]:
sentence_length = np.prod(cfg.T)
start = time.time()
config = GPTConfig(vocab_size=cfg.ns[-1], n_embd=384, n_head=6, n_layer=6)
m = GPT(config)
m = nn.DataParallel(m)
m.to(config.device)

number of parameters: 10.64M


DataParallel(
  (module): GPT(
    (transformer): ModuleDict(
      (wte): Embedding(10, 384)
      (wpe): Embedding(256, 384)
      (drop): Dropout(p=0.0, inplace=False)
      (h): ModuleList(
        (0-5): 6 x Block(
          (ln_1): LayerNorm()
          (attn): MultiHeadAttention(
            (heads): ModuleList(
              (0-5): 6 x Head(
                (key): Linear(in_features=384, out_features=64, bias=False)
                (query): Linear(in_features=384, out_features=64, bias=False)
                (value): Linear(in_features=384, out_features=64, bias=False)
                (dropout): Dropout(p=0.0, inplace=False)
              )
            )
            (proj): Linear(in_features=384, out_features=384, bias=True)
            (dropout): Dropout(p=0.0, inplace=False)
          )
          (ln_2): LayerNorm()
          (mlp): MLP(
            (c_fc): Linear(in_features=384, out_features=1536, bias=True)
            (gelu): GELU(approximate='none')
            (c_proj)

In [55]:
 # print the number of parameters in the model
print(sum(p.numel() for p in m.parameters()) / 1e6, "M parameters")

10.742784 M parameters


In [56]:
# data loading = sample new sentences to fill-in the mini-batch
def get_batch(config: GPTConfig = GPTConfig()):
    sentence = cfg.sample_flattened(1)[0][0].view(sentence_length)  # reshape in a 1d tensor
    # generate a small batch of data of inputs x and targets y
    ix = torch.randint(0, sentence_length - config.block_size, size=(config.batch_size,))
    x = torch.stack([sentence[i: i + config.block_size] for i in ix])
    y = torch.stack([sentence[i+1: i + config.block_size + 1] for i in ix])
    x, y = x.to(config.device), y.to(config.device)
    return x, y

In [57]:
@torch.no_grad()
def estimate_loss(eval_iters):
    # This function samples a new batch of sentences and evaluates the loss of the model
    out = {}
    m.eval()
    losses = torch.zeros(eval_iters)
    for k in range(eval_iters):
        X, Y = get_batch()
        logits = m(X)
        loss = nn.functional.cross_entropy(logits.view(-1, logits.size(-1)), Y.view(-1), ignore_index=-1)
        losses[k] = loss.item()
    out["val"] = losses.mean()
    m.train()
    return out

In [58]:
context_length = 3
@torch.no_grad()
def estimate_grammar_err(n_gen=100):
    m.eval()
    model = m.module
    # generate n_gen sentences from the model and check their correctness
    # for generating sentences from the model, we first sample a real sentence from the grammar
    # then, the model is given the first 'context_length' tokens and asked to complete the sentence
    # Returns the number of sentence correct (with 0 mistake) at each level
    error_per_sentence = []    
    for i in range(n_gen):
        mistakes = []
        context = cfg.sample_flattened(1)[0][0][:,:context_length].to(config.device)
        gen_sentence = m.module.generate(context.reshape(1,3), max_new_tokens=sentence_length-context_length)[0].view(-1,1)
        _, err = cfg.collapse_and_get_err(gen_sentence.view(*cfg.T).cpu())
        for level_errors in err:
            mistakes.append(torch.count_nonzero(level_errors).detach().numpy())
        error_per_sentence.append(np.array(mistakes))
    error_per_sentence = np.array(error_per_sentence)
    # compute number of sentence that are correct at each level of the grammar
    res = []
    for l in range(len(cfg.T)):
        nb_correct = (n_gen - np.count_nonzero(error_per_sentence[:,l]))
        res.append(nb_correct)
    m.train()
    return np.array(res)

In [59]:
training_parameters = {'max_iters' : 5000,
                       'eval_interval' : 200,
                       'eval_iters' : 50,
                       'quality_metric_iters' : 40,
                       'learning_rate' : 1e-3,
                       'architecture':"GPT 10.7M",
                       'grammar': cfg.__str__(),
                       'batch_size':config.batch_size,}
training_parameters['optimizer'] = torch.optim.AdamW(m.parameters(), lr=training_parameters['learning_rate'])
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(training_parameters['optimizer'], mode='min', patience=2, factor=0.1) # Divide lr by 10

In [60]:
# Training loop
def train(m):
    for iter in range(training_parameters['max_iters']):
        # every once in a while evaluate the loss on newly generated sentences
        if iter % training_parameters['eval_interval'] == 0 or iter == training_parameters['max_iters'] - 1:
            wandb.log({"nb sentences seen": iter*config.batch_size})
            val_loss = estimate_loss(training_parameters['eval_iters'])['val']
            print(
                f"step {iter}: val loss {val_loss:.4f}"
            )
            wandb.log({"loss": val_loss})
            scheduler.step(metrics=val_loss)
            wandb.log({"learning_rate": training_parameters['optimizer'].param_groups[0]["lr"]})
            
            errors = estimate_grammar_err(training_parameters['quality_metric_iters'])
            print(
                f"step {iter}: correct sentences for each level{errors}"
            )
            for i, err in enumerate(errors):
                wandb.log({f'% of correct sentences at level {i}': err/training_parameters['quality_metric_iters']})
    
        # sample a batch of data
        xb, yb = get_batch()
    
        # evaluate the loss
        logits = m(xb)
        training_parameters['optimizer'].zero_grad(set_to_none=True)
        loss = nn.functional.cross_entropy(logits.view(-1, logits.size(-1)), yb.view(-1), ignore_index=-1)
        loss.backward()
        training_parameters['optimizer'].step()

In [61]:
wandb.init(project='CFG-experiments',config=training_parameters)
wandb.watch(m, log='all', log_freq=1)

train(m)
wandb.finish()

% of correct sentences at level 0,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
% of correct sentences at level 1,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
% of correct sentences at level 2,▁▁▁▁▁▄▂▂▄▅▄▄▆▇██
learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
loss,█▅▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁
% of correct sentences at level 0,0.0
% of correct sentences at level 1,0.0
% of correct sentences at level 2,0.775
learning_rate,0.001
loss,0.14453


step 0: val loss 2.3887
step 0: correct sentences for each level[0 0 0]
step 200: val loss 0.8433
step 200: correct sentences for each level[0 0 0]
step 400: val loss 0.3113
step 400: correct sentences for each level[0 0 0]
step 600: val loss 0.2171
step 600: correct sentences for each level[0 0 3]
step 800: val loss 0.2052
step 800: correct sentences for each level[0 0 4]
step 1000: val loss 0.1902
step 1000: correct sentences for each level[0 0 7]
step 1200: val loss 0.1825
step 1200: correct sentences for each level[0 0 6]
step 1400: val loss 0.1800
step 1400: correct sentences for each level[0 0 5]
step 1600: val loss 0.1686
step 1600: correct sentences for each level[ 0  0 12]
step 1800: val loss 0.1654
step 1800: correct sentences for each level[ 0  0 13]
step 2000: val loss 0.1643
step 2000: correct sentences for each level[0 0 2]
step 2200: val loss 0.1565
step 2200: correct sentences for each level[ 0  0 27]
step 2400: val loss 0.2061
step 2400: correct sentences for each leve

% of correct sentences at level 0,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
% of correct sentences at level 1,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
% of correct sentences at level 2,▁▁▁▂▂▂▂▂▃▃▁▆▁▅▄▅▅▅▆▇▆█▇▇▆▃
learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
loss,█▃▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
% of correct sentences at level 0,0.0
% of correct sentences at level 1,0.0
% of correct sentences at level 2,0.25
learning_rate,0.001
loss,0.13956
